In [1]:
import nltk
from collections import defaultdict
from difflib import SequenceMatcher
from nltk.util import ngrams

# Tokenization
def tokenize(text):
    return nltk.word_tokenize(text)

# Inverted Index Construction
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    for doc_id, text in enumerate(documents):
        tokens = tokenize(text)
        for token in tokens:
            inverted_index[token].append(doc_id)
    return inverted_index

# Levenshtein Distance
def levenshtein_distance(word1, word2):
    print(SequenceMatcher(None, word1, word2).ratio())
    return SequenceMatcher(None, word1, word2).ratio()

# Spell Correction using N-Grams
def generate_ngrams(word, n=3):
    return list(ngrams(word, n))

def spell_correction_ngram(word, vocabulary):
    best_match = None
    min_distance = float('inf')
    for vocab_word in vocabulary:
        distance = levenshtein_distance(word, vocab_word)
        if distance < min_distance:
            min_distance = distance
            best_match = vocab_word
    return best_match

# Soundex Algorithm for Phonetic Spell Correction
def soundex(word):
    """Returns the soundex code for the word."""
    word = word.upper()
    soundex_code = word[0]

    soundex_mapping = {
        "BFPV": "1", "CGJKQSXZ": "2", "DT": "3", "L": "4",
        "MN": "5", "R": "6"
    }

    for char in word[1:]:
        for key in soundex_mapping.keys():
            if char in key:
                code = soundex_mapping[key]
                if code != soundex_code[-1]:  # Avoid duplicates
                    soundex_code += code
                break
        if len(soundex_code) == 4:
            break

    soundex_code = (soundex_code + "000")[:4]  # Pad with zeros if necessary
    return soundex_code

def spell_correction_soundex(word, vocabulary):
    """Corrects spelling using Soundex codes."""
    word_soundex = soundex(word)
    for vocab_word in vocabulary:
        if soundex(vocab_word) == word_soundex:
            return vocab_word
    return word

# Example Usage

documents = [
    "I love programming in Python",
    "Phonetic spell correction is interesting",
    "Soundex is a useful algorithm for spelling"
]

# Tokenize and build inverted index
tokens = tokenize(documents[0])
inverted_index = build_inverted_index(documents)

# Test Spell Correction
vocabulary = list(inverted_index.keys())
word = "phonetick"
corrected_ngram = spell_correction_ngram(word, vocabulary)
corrected_soundex = spell_correction_soundex(word, vocabulary)

print(f"N-Gram Corrected Word: {corrected_ngram}")
print(f"Soundex Corrected Word: {corrected_soundex}")


0.0
0.3076923076923077
0.3
0.18181818181818182
0.4
0.8235294117647058
0.2857142857142857
0.21052631578947367
0.18181818181818182
0.4
0.375
0.0
0.13333333333333333
0.1111111111111111
0.16666666666666666
0.23529411764705882
N-Gram Corrected Word: I
Soundex Corrected Word: Phonetic


## TOKENIZATION AND INVERTED INDEX FROM ABOVE FUNCTIONS

In [2]:
tokens

['I', 'love', 'programming', 'in', 'Python']

In [3]:
inverted_index

defaultdict(list,
            {'I': [0],
             'love': [0],
             'programming': [0],
             'in': [0],
             'Python': [0],
             'Phonetic': [1],
             'spell': [1],
             'correction': [1],
             'is': [1, 2],
             'interesting': [1],
             'Soundex': [2],
             'a': [2],
             'useful': [2],
             'algorithm': [2],
             'for': [2],
             'spelling': [2]})

In [4]:
# Levenshtein Distance Example
word1 = "kitten"
word2 = "sitting"
lev_distance = levenshtein_distance(word1, word2)

In [5]:
lev_distance

0.6153846153846154

## Edit Distance DP

In [6]:
# Dynamic Programming approach for Levenshtein Distance
def levenshtein_distance_dp(word1, word2):
    # Create a 2D table to store the distances
    m, n = len(word1), len(word2)
    
    # Initialize the table with 0 values
    dp = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    # Fill the first row and column with indices (base cases)
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Compute the distance using the recurrence relation
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                cost = 0
            else:
                cost = 1
            dp[i][j] = min(dp[i - 1][j] + 1,    # Deletion
                           dp[i][j - 1] + 1,    # Insertion
                           dp[i - 1][j - 1] + cost)  # Substitution

    # The last cell contains the final Levenshtein distance
    return dp[m][n], dp

# Example usage:
word1 = "kitten"
word2 = "sitting"

lev_distance, dp_table = levenshtein_distance_dp(word1, word2)

# Output
print(f"Levenshtein Distance between '{word1}' and '{word2}': {lev_distance}")

# Optional: print the DP table for better understanding
print("\nDP Table:")
for row in dp_table:
    print(row)

Levenshtein Distance between 'kitten' and 'sitting': 3

DP Table:
[0, 1, 2, 3, 4, 5, 6, 7]
[1, 1, 2, 3, 4, 5, 6, 7]
[2, 2, 1, 2, 3, 4, 5, 6]
[3, 3, 2, 1, 2, 3, 4, 5]
[4, 4, 3, 2, 1, 2, 3, 4]
[5, 5, 4, 3, 2, 2, 3, 4]
[6, 6, 5, 4, 3, 3, 2, 3]


## N-Grams

In [9]:
from nltk.util import ngrams

# Function to generate n-grams and display the steps
def generate_ngrams_visualization(text, n):
    tokens = list(text)  # Tokenize the word into individual characters
    print(f"\nOriginal Text: {text}")
    print(f"Generating {n}-grams:")
    
    # Generate the n-grams
    ngrams_list = list(ngrams(tokens, n))
    # Display the steps
    for i, ng in enumerate(ngrams_list):
        print(f"Step {i + 1}: {ng}")

    return ngrams_list

# Example Usage
word = "hello"
n = 3
ngrams_visualized = generate_ngrams_visualization(word, n)

# Display the final result
print(f"\n{n}-grams for the word '{word}': {ngrams_visualized}")



Original Text: hello
Generating 3-grams:
Step 1: ('h', 'e', 'l')
Step 2: ('e', 'l', 'l')
Step 3: ('l', 'l', 'o')

3-grams for the word 'hello': [('h', 'e', 'l'), ('e', 'l', 'l'), ('l', 'l', 'o')]


## SOUNDEX

In [10]:
# Soundex Algorithm with Adjacent Duplicate Removal and Zero Removal
def soundex_step_by_step(word):
    word = word.upper()  # Convert to uppercase
    soundex_code = word[0]  # The first character remains as is

    # Mapping of letters to Soundex digits
    soundex_mapping = {
        "BFPV": "1", "CGJKQSXZ": "2", "DT": "3", "L": "4",
        "MN": "5", "R": "6"
    }
    
    print(f"Step 1: Keep the first letter -> {soundex_code}")
    
    # Generate the Soundex code
    for char in word[1:]:
        for key in soundex_mapping.keys():
            if char in key:
                code = soundex_mapping[key]
                soundex_code += code
                print(f"Convert '{char}' to {code} -> {soundex_code}")
                break
        else:
            print(f"Skip '{char}' (not in mapping) -> {soundex_code}")
    
    # Step 2: Remove adjacent duplicates
    filtered_code = soundex_code[0]  # First character remains unchanged
    for i in range(1, len(soundex_code)):
        if soundex_code[i] != soundex_code[i - 1]:
            filtered_code += soundex_code[i]

    print(f"Step 2: Remove adjacent duplicates -> {filtered_code}")
    
    # Step 3: Remove any zeros
    filtered_code = filtered_code.replace('0', '')
    print(f"Step 3: Remove '0's -> {filtered_code}")

    # Step 4: Pad with zeros if the code is shorter than 4 characters
    final_code = (filtered_code + "000")[:4]
    print(f"Step 4: Pad with zeros -> {final_code}")
    
    return final_code

# Example Usage
word = "Smith"
soundex_code = soundex_step_by_step(word)

print(f"\nFinal Soundex Code for '{word}': {soundex_code}")


Step 1: Keep the first letter -> S
Convert 'M' to 5 -> S5
Skip 'I' (not in mapping) -> S5
Convert 'T' to 3 -> S53
Skip 'H' (not in mapping) -> S53
Final step: Pad with zeros -> S530

Final Soundex Code for 'Smith': S530
